In [96]:
import os
from copy import deepcopy

import dash
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
import pandas as pd
import plotly.express as px
import torch
from dash.dependencies import Input, MATCH, Output, State
from jupyter_dash import JupyterDash as Dash
from plotly import graph_objects as go
from shapely.geometry import MultiPolygon, Point, LineString, Polygon

from MPNet.enet import data_loader as ae_dl
from MPNet.enet.CAE import ContractiveAutoEncoder
from MPNet.neuralplanner import feasibility_check, lvc
from MPNet.pnet.data_loader import loader
from MPNet.pnet.model import PNet

os.getcwd()

'/home/jhonas/dev/mpnet'

In [97]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash("Replanning", external_stylesheets=external_stylesheets)

stage_figures = {0: [go.Figure(layout=dict(title="Bidirectional Planning"))],
                 1: [go.Figure(layout=dict(title="States Connection"))],
                 2: [go.Figure(layout=dict(title="LVC"))],
                 3: [go.Figure(layout=dict(title="Replanning"))],
                 4: [go.Figure(layout=dict(title="Final Path"))]}

graphs = {i: dcc.Graph(id={"type": "graph", "index": i}, animate=i not in (2, 3, 4)) for i in stage_figures.keys()}

manual_update = dcc.Interval(id="update_interval", disabled=True, n_intervals=0, interval=1000, max_intervals=1)
slide_update = dcc.Interval(id="slide_interval", disabled=True, n_intervals=0, interval=1200, max_intervals=-1)
pause_button = html.Button("PAUSE", id="pause_but", n_clicks=0,
                           style={"width"     : "100px", "textAlign": "center", "marginBottom": "20px",
                                  "marginLeft": '20px'})
feasible_buttons = {i: html.Button("FEASIBLE", id={"type": "feasible_but", "index": i}, n_clicks=0, disabled=True,
                                   style={"textAlign": "center", "padding": "0 15px", "font-size": "14px", 'color':
                                       "white", "background": "red", "cursor": "default", "font-weight": "bold"}) for i
                    in stage_figures.keys()}

app.layout = html.Div([
        html.H2("MPNet Execution Step by Step"),
        html.Div([
                html.Button("PLAY", id="play_but", n_clicks=0,
                            style={"width": "100px", "textAlign": "center", "marginBottom": "20px"}),
                pause_button,
                html.Div(id="feasible_div",
                         style={"width"  : "125px", "marginBottom": "20px", "float": "right", "display": "flex",
                                "padding": "0 15px", "justify-content": "flex-end"}),
                dcc.Slider(id="step_slider", value=0, min=0, max=4, step=1, marks={
                        0: "Bidirectional",
                        1: "States Connection",
                        2: "LVC",
                        3: "Replanning",
                        4: "Final Path"
                        })], style={"margin-bottom": "25px", "margin-left": "40px", "margin-right": "40px"}),
        html.Div(id="hidden_slider"),
        html.Div(id="graph_div", style={"display": "flex", "justify-content": "center"}),
        slide_update,
        manual_update,
        ])

frame_sliders = {0: dcc.Slider(id={"type": "dynamic_slider", "index": 0}, min=1, max=2, value=1, step=1),
                 1: dcc.Slider(id={"type": "dynamic_slider", "index": 1}, min=1, max=2, value=1, step=1),
                 2: dcc.Slider(id={"type": "dynamic_slider", "index": 2}, min=1, max=2, value=1, step=1),
                 3: dcc.Slider(id={"type": "dynamic_slider", "index": 3}, min=1, max=2, value=1, step=1),
                 4: dcc.Slider(id={"type": "dynamic_slider", "index": 4}, min=1, max=2, value=1, step=1)}


@app.callback(Output({"type": "graph", "index": MATCH}, "figure"),
              Input({"type": "dynamic_slider", "index": MATCH}, "value"), Input("step_slider", "value"))
def master_slide_cb(value, master_value):
    #     return dash.no_update, f'{value} and {master_value}'
    try:
        return stage_figures[master_value][value - 1]
    except IndexError:
        return dash.no_update


@app.callback(Output({"type": "dynamic_slider", "index": MATCH}, "value"), Input("slide_interval", "n_intervals"),
              State("hidden_slider", "children"))
def interval_update(value, div_slider_children):
    max_steps = div_slider_children[0]['props']['max']
    if value == max_steps - 1:
        slide_update.n_intervals = 0
    return int(value % max_steps + 1)


@app.callback(Output("slide_interval", "disabled"), Input("play_but", "n_clicks"), Input("pause_but", "n_clicks"),
              Input("slide_interval", "n_intervals"), State("hidden_slider", "children"), prevent_initial_call=True)
def set_slide_interval_state(p_but, pause_but, value, children):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if 'play' in changed_id:
        return False
    elif 'pause' in changed_id:
        return True
    elif 'slide' in changed_id:
        max_interval = children[0]['props']['max']
        if value % max_interval == max_interval - 1:
            return True
        else:
            return dash.no_update


@app.callback(Output({"type": "feasible_but", "index": MATCH}, "style"),
              Input({"type": "feasible_but", "index": MATCH}, "n_clicks"))
def change_feasible_button_style(value):
    mapping = {False: "red", True: "green"}
    return {"width" : "125px", "textAlign": "center", "color": "white", "background": mapping[bool(value)],
            "cursor": "default", "font-weight": "bold", "font-size": "14px", "padding": "0 15px"}


@app.callback([Output("hidden_slider", "children"), Output("hidden_slider", "style"), Output("graph_div", "children"),
               Output("feasible_div", "children")],
              [Input("step_slider", "value"), Input("update_interval", "n_intervals")])
def show_secondary_slider(value, enabled):
    return [frame_sliders[value]], {"marginBottom": "15px", "marginLeft": "40px", "marginRight": "40px"}, [
            graphs[value]], feasible_buttons[value]



In [98]:
def update_sliders(figure_idx, markers={}):
    frame_sliders[figure_idx].marks = markers
    frame_sliders[figure_idx].max = len(markers)


def update_figures(stage_idx, figures=[]):
    stage_figures[stage_idx] = figures


def update_feasible(analised_path, env, stage_idx):
    feasible = feasibility_check(analised_path, env)
    feasible_buttons[stage_idx].n_clicks = int(feasible)

Initialize both modules from previously trained checkpoints

In [99]:
pnet = PNet.load_from_checkpoint('models/pnet_3_cae.ckpt')
enet_ckpt = pnet.training_config['enet']
enet = ContractiveAutoEncoder.load_from_checkpoint(enet_ckpt)
pnet.freeze()
enet.freeze()

Load the environment and trajectory data.
The `loader` funcion returns a `torch.Dataset` instead of a `torch.DataLoader` when used with the get_dataset option
set to True.


In [100]:
environments = ae_dl.load_perms(110, 0)

# This may take a while
data = loader(enet, "valEnv/", 110, 0, 1, get_dataset=True)


127.0.0.1 - - [16/May/2021 20:00:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 20:00:25] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [16/May/2021 20:00:25] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [16/May/2021 20:00:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 20:00:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2021 20:00:26] "POST /_dash-update-component HTTP/1.1" 204 -



When acessing the dataset with a tuple where the second element is a boolean set to True, the return value is a
tuple with ` (input, target, path, embed_idx)`, where input is the processed input of the pnet module (where the
environment has already been processed by the enet module), target is the target to be used when training, path is
the reference path where the trajectory path comes from (only used for ilustration) and embed_idx is the index of
the environment in the environments list.

In [101]:
# Choose a path to be processed from available data
path_idx = np.random.choice(len(data), 1)[0]
path_idx = 104160
model_input, _, reference_path, env_idx = data[path_idx, True]
data[path_idx, True], path_idx

((tensor([ 1.6291,  1.0762,  2.7489,  1.5210, -0.0190, -0.0365, -0.3569,  2.7006,
           3.2458, -0.3456,  0.3670,  2.4598, -0.5975,  0.7302,  1.5588,  0.8238,
           1.3224,  1.6739,  0.2004,  0.9638,  1.0873, -0.6527, -0.4027,  2.1710,
           0.9183, -1.4306,  0.6946,  0.7743,  4.8255, -6.0217,  8.5704, -6.0862]),
  tensor([ 5.8086, -6.0407]),
  array([[-16.31999969,  -1.24000001],
         [-12.73313221,  -2.2647926 ],
         [-11.00150608,  -2.77480362],
         [ -8.91285181,  -3.32947919],
         [ -6.84361193,  -3.88879029],
         [ -5.34212522,  -4.26410723],
         [ -1.57698667,  -5.38536983],
         [ -0.49412161,  -5.68837728],
         [  2.78100487,  -6.0127461 ],
         [  4.82552458,  -6.02169821],
         [  5.80856036,  -6.04067165],
         [  8.57040909,  -6.08615535]]),
  100),
 104160)

Since the `model_input` variable refers to a single step in a trajectory, it is necessary to change the elements that
refer to the current state to the origin state of said trajectory.


In [102]:
model_input[-4:-2] = torch.from_numpy(reference_path[0])

Using the `env_idx` variable, it is possible to retrieve the environment data from the `environments` list.


In [ ]:
def draw_obstacles(fig, perm, **kwargs):
    obstacles = []
    for obstacle in perm:
        x, y = obstacle
        obstacles.extend([[x - 2.5, y - 2.5], [x - 2.5, y + 2.5], [x + 2.5, y + 2.5], [x + 2.5, y - 2.5],
                          [x - 2.5, y - 2.5], [None, None]])
    obstacles = np.array(obstacles)
    x = obstacles[:, 0]
    y = obstacles[:, 1]
    fig.add_trace(go.Scatter(x=x, y=y, fill="toself", fillcolor="black", name="obstacle"), **kwargs)

In [103]:
environment = environments[env_idx]
obstacles_trace, obstacles = draw_obstacles(None, environment, return_trace=True, marker=dict(color='black'))
obstacle = []
obstacles_polygon = []
for point in obstacles:
    if None in point:
        polygon = Polygon(obstacle)
        obstacles_polygon.append(polygon)
        obstacle = []
    else:
        obstacle.append(tuple(point))
env_polygons = MultiPolygon(obstacles_polygon)

In [104]:
def is_in_collision(x, env):
    x = Point(x)
    for obstacle in env:
        if obstacle.contains(x):
            return True
    return False


def steer_to(start, end, env):
    start = Point(start)
    end = Point(end)
    line = LineString([start, end])
    for polygon in env:
        if polygon.intersects(line):
            return False
    else:
        return True


def feasibility_check(path, env) -> bool:
    for i in range(0, len(path[:-1])):
        ind = steer_to(path[i], path[i + 1], env)
        if not ind:
            return False
    return True


def lvc(path, env):
    # Iterate from the first beacon state to the second to last.
    for i in range(0, len(path) - 1):
        # Iterate from the last beacon state to the ith.
        for j in range(len(path) - 1, i + 1, -1):
            ind = steer_to(path[i], path[j], env)
            if ind:
                pc = []
                for k in range(0, i + 1):
                    pc.append(path[k])
                for k in range(j, len(path)):
                    pc.append(path[k])

                return lvc(pc, env)

    return path


def h_lvc(path, env):
    p_1 = path[0]
    if is_in_collision(p_1, env):
        return False, None
    optimized_path = [p_1]
    for j, p_2 in enumerate(path[::-1][:-1]):
        steerable = steer_to(p_1, p_2, env)
        temp_path = []
        if steerable:
            feasible, states = h_lvc(path[len(path) - j - 1:], env)
            if feasible:
                optimized_path.extend(states)
                return True, optimized_path
    return False, None





Now we define the planning function and some auxiliary functions.


In [105]:
def bidirectional_planning(pnet, origin, goal, env):
    result_1 = deepcopy(origin[-4:-2])
    result_2 = deepcopy(goal[-4:-2])
    path_1, path_2 = [result_1.numpy()], [result_2.numpy()]
    tree, target_reached = False, False
    step = 0
    while not target_reached and step < 150:
        step += 1
        if not tree:
            result_1 = pnet(origin)
            result_1 = result_1.data.detach()
            path_1.append(result_1.numpy())
            origin[-4:-2] = result_1
            goal[-2:] = result_1
        else:
            result_2 = pnet(goal)
            result_2 = result_2.data.detach()
            path_2.append(result_2.numpy())
            goal[-4:-2] = result_2
            origin[-2:] = result_2
        tree = not tree
        target_reached = steer_to(result_1.numpy(), result_2.numpy(), env)
    return target_reached, path_1, path_2


def plan(pnet, env, model_input):
    origin = deepcopy(model_input)
    goal = deepcopy(origin)
    goal[-4:] = goal[[-2, -1, -4, -3]]
    target_reached, path_1, path_2 = bidirectional_planning(pnet, origin, goal, env)
    return target_reached, path_1, path_2


# Checks if it's necessary to replan this section
def bidirectional_replan_check(pnet, env, start_point, goal_point, model_input):
    start = deepcopy(model_input)
    start[-4:] = torch.as_tensor([*start_point, *goal_point])
    goal = deepcopy(start)
    goal[-4:] = goal[[-2, -1, -4, -3]]
    steerable = steer_to(start_point, goal_point, env)
    return steerable, start, goal

In [106]:
target_reached, path_1, path_2 = plan(pnet, env_polygons, model_input)

Animation of all the states processed by the bidirectional planning step.

In [107]:
frames = pd.DataFrame(columns=["Frame", "x", "y"])
obstacles_df = pd.DataFrame(data=obstacles, columns=['x', 'y'])

path_1 = np.array(path_1)
path_2 = np.array(path_2)

points_1x, points_1y = [], []
points_2x, points_2y = [], []

for n in range(len(path_1)):
    frames = pd.concat(
            [frames, pd.DataFrame([[n, *p_1, '1', 'Bidirectional'] for p_1 in path_1[:n + 1]],
                                  columns=['Frame', 'x', 'y', 'Planner', 'Step'])], ignore_index=True)
    frames = pd.concat(
            [frames, pd.DataFrame([[n, *p_2, '2', 'Bidirectional'] for p_2 in path_2[:n + 1]],
                                  columns=['Frame', 'x', 'y', 'Planner', 'Step'])], ignore_index=True)

In [108]:
def add_obstacles_trace(figure):
    trace = figure.select_traces(selector=dict(name='obstacle'))
    if not list(trace):
        figure.add_trace(obstacles_trace)
    traces = figure.data[:-1]
    obstacles = figure.data[-1]
    figure.data = [obstacles, *traces]


def update_fig(figure):
    figure.update_layout(dict(width=1280, height=720, legend=dict(yanchor="bottom", xanchor="left", y=-0.1, x=-0.2)))
    figure.update_traces(marker=dict(size=15),
                         selector=dict(mode='markers'))

In [109]:
unique_frames = frames[frames['Step'] == 'Bidirectional'].Frame.unique()
figure_0 = [px.scatter(frames[(frames.Step == 'Bidirectional') & (frames.Frame == f)], x="x", y="y", color='Planner',
                       range_x=[-20, 20], range_y=[-20, 20]) for f in unique_frames]
for fig in figure_0:
    add_obstacles_trace(fig)
    update_fig(fig)

update_sliders(0, {i + 1: f"{i + 1}" for i, _ in enumerate(figure_0)})
update_figures(0, figure_0)
manual_update.disabled = False


Connecting the beacon states.

In [110]:
path = np.concatenate([path_1, path_2[::-1]])

frames = pd.concat(
        [frames,
         pd.DataFrame([[0, *p, "", "Connection"] for p in path], columns=['Frame', 'x', 'y', 'Planner', 'Step']),
         pd.DataFrame([[1, *p, "", "Connection"] for p in path], columns=['Frame', 'x', 'y', 'Planner', 'Step'])],
        ignore_index=True)

In [111]:
unique_frames = frames[frames['Step'] == 'Connection'].Frame.unique()
figure_1 = [px.scatter(frames[(frames.Step == 'Connection') & (frames.Frame == f)], x="x", y="y", color='Planner',
                       range_x=[-20, 20], range_y=[-20, 20]) for f in unique_frames]
for fig in figure_1:
    add_obstacles_trace(fig)
    update_fig(fig)

figure_1[-1].update_traces(dict(mode='lines+markers', marker=dict(size=15)), selector=dict(mode='markers'))
update_sliders(1, {i + 1: f"{i + 1}" for i, _ in enumerate(figure_1)})
update_figures(1, figure_1)
update_feasible(path, env_polygons, 1)
manual_update.disabled = False


Running the Lazy Vertex Contraction algorithm to resume the obtained trajectory.

In [112]:

frames = pd.concat(
        [frames,
         pd.DataFrame([[0, *p, "", "lvc"] for p in path], columns=['Frame', 'x', 'y', 'Planner', 'Step'])],
        ignore_index=True)

path = np.array(lvc(path, env_polygons))

frames = pd.concat(
        [frames,
         pd.DataFrame([[1, *p, "", "lvc"] for p in path], columns=['Frame', 'x', 'y', 'Planner', 'Step'])],
        ignore_index=True)

In [113]:
unique_frames = frames[frames['Step'] == 'lvc'].Frame.unique()
figure_2 = [px.scatter(frames[(frames.Step == 'lvc') & (frames.Frame == f)], x="x", y="y", color='Planner',
                       range_x=[-20, 20], range_y=[-20, 20]) for f in unique_frames]
for fig in figure_2:
    add_obstacles_trace(fig)
    update_fig(fig)
    fig.update_traces(dict(mode='lines+markers', marker=dict(size=15)), selector=dict(mode='markers'))

update_sliders(2, {i + 1: f"{i + 1}" for i, _ in enumerate(figure_2)})
update_figures(2, figure_2)
update_feasible(path, env_polygons, 2)
manual_update.disabled = False

Remove beacon states that colide with an obstacle.

In [114]:
def remove_invalid_beacon_states(path):
    updated_index = []
    new_path = []
    for state in path:
        if not is_in_collision(state, env_polygons):
            new_path.append(state)
        else:
            new_path[-1] = np.mean([new_path[-1], new_path[-2]], axis=0)
    for n, state in enumerate(new_path[::-1]):
        if is_in_collision(state, env_polygons):
            try:
                new_path[n - 1] = np.mean([new_path[n - 1], new_path[n - 2]], axis=0)
            except IndexError:
                pass
    new_path = np.array(new_path)
    return new_path


new_path = remove_invalid_beacon_states(path)

# For illustration purposes only
connectable_paths = []
temp_list = [path[0]]
for i in range(1, len(path)):
    try:
        if steer_to(path[i], path[i - 1], env_polygons):
            temp_list.append(path[i])
        elif steer_to(path[i], path[i + 1], env_polygons):
            connectable_paths.append(temp_list)
            temp_list = [path[i]]
        else:
            if temp_list:
                connectable_paths.append(temp_list)
                temp_list = []
    except IndexError:
        if temp_list:
            connectable_paths.append(temp_list)
            temp_list = []
else:
    if not temp_list:
        temp_list = [path[-1]]
    connectable_paths.append(temp_list)

frames = pd.concat([frames,
                    pd.DataFrame([["Previous Path", *p, "Optimized Path", "Replan"] for p in path],
                                 columns=['Frame', 'x', 'y', 'Planner', 'Step']),
                    *[pd.DataFrame([["Valid Beacon States", *p, f"Valid Subtrajectory {n}", "Replan"] for p in stage],
                                   columns=["Frame", "x", "y", "Planner", "Step"]) for n, stage in
                      enumerate(connectable_paths)]], ignore_index=True)


In [115]:
replanned_path = path
tries = 0
feasible = feasibility_check(new_path, env_polygons)
while not feasible and tries < 20:
    tries += 1
    new_path = remove_invalid_beacon_states(new_path)
    replanned_path = [new_path[0]]
    for i in range(len(new_path) - 1):
        steerable, start, goal = bidirectional_replan_check(pnet, env_polygons, new_path[i], new_path[i + 1],
                                                            model_input)
        if steerable:
            replanned_path.append(new_path[i + 1])
        else:
            target_reach, rpath_1, rpath_2 = bidirectional_planning(pnet, start, goal, env_polygons)
            replanned_path = list(np.concatenate([replanned_path, rpath_1, rpath_2[::-1]]))

    replanned_path = list(np.unique(replanned_path, axis=0))
    lvc_replanned_path = lvc(replanned_path, env_polygons)
    lvc_replanned_path = np.array(lvc_replanned_path)
    feasible = feasibility_check(lvc_replanned_path, env_polygons)
    if feasible:
        new_path = lvc_replanned_path
        break
    else:
        new_path = np.array(replanned_path)
    frames = pd.concat(
            [frames, pd.DataFrame(
                    [[f"Try {tries}", *beacon_state, f"Replanned Path {tries}", "Replan"] for beacon_state in
                     new_path],
                    columns=["Frame", "x", "y", "Planner", "Step"])], ignore_index=True)

In [116]:

new_path = np.array(lvc(new_path, env_polygons))
frames = pd.concat([frames,
                    pd.DataFrame(
                            [["Bidirectional Replanning", *beacon_state, "Replanned Path", "Replan"] for beacon_state in
                             new_path], columns=["Frame", "x", "y", "Planner", "Step"])], ignore_index=True)

In [117]:
unique_frames = frames[frames['Step'] == 'Replan'].Frame.unique()
figure_3 = [px.scatter(frames[(frames.Step == 'Replan') & (frames.Frame == f)], x="x", y="y", color='Planner',
                       range_x=[-20, 20], range_y=[-20, 20]) for f in unique_frames]
for fig in figure_3:
    add_obstacles_trace(fig)
    update_fig(fig)
    fig.update_traces(dict(mode='lines+markers', marker=dict(size=15)), selector=dict(mode='markers'))

update_sliders(3, {i + 1: {'label': f} for i, f in enumerate(unique_frames)})
update_figures(3, figure_3)
update_feasible(new_path, env_polygons, 3)
manual_update.disabled = False

In [118]:
figure_4 = []
for k, v in stage_figures.items():
    if k != 4:
        figure_4.append(v[-1])

unique_steps = frames.Step.unique()

update_sliders(4, {n + 1: f for n, f in enumerate(unique_steps)})
update_figures(4, figure_4)
update_feasible(new_path, env_polygons, 4)
manual_update.disabled = False

In [119]:
app.run_server(mode='external', debug=False)


/home/jhonas/dev/mpr/.venv/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

127.0.0.1 - - [16/May/2021 20:00:30] "GET /_shutdown_b4450afd-37b7-4cd9-bf50-51f5ac47dd27 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/May/2021 20:00:31] "GET /_alive_b4450afd-37b7-4cd9-bf50-51f5ac47dd27 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/
